In [123]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [124]:
df = pd.read_csv("data/new_era5_data.csv",sep=";")
#Read and load data from csv file

df.time = pd.to_datetime(df.time)
#Convert string to datetime

df = df[["time","tanom_cao","cao_area_weighted","t2min_weighted"]]

In [125]:
#Set up start and end date of each year, deletes March 31st if leap year

#Set first day of the year (Nov-01)
day_first = pd.date_range("{}-11-01".format(np.min(df.time.dt.year)), "{}-12-01".format(np.max(df.time.dt.year)), freq='12MS')
#Find leap years
is_leap = day_first.is_leap_year
#Set last day of the year (March-31)
day_last = pd.date_range("{}-03-01".format(np.min(df.time.dt.year)), "{}-03-31".format(np.max(df.time.dt.year)), freq='12ME', inclusive='right')
#If leap year, remove the last day
dt1 = day_last[~is_leap]
dt2 = day_last[is_leap] - timedelta(days=1)
#Creates a series with the last date of each year
day_last = pd.concat([pd.Series(dt1), pd.Series(dt2)],ignore_index=True)
day_last = day_last.sort_values(ignore_index=True)

df_times = pd.DataFrame({'day_first': pd.Series(day_first),'day_last': pd.Series(day_last).shift(-1)})
df_times["year"] = pd.Series(df_times.day_last.dt.year)
df_times = df_times.iloc[0:-1] #exclude last row
df["year"] = np.nan
for row in zip(df_times.day_first,df_times.day_last,df_times.year):
    #print(row)
    df.loc[(df['time'] >= row[0]) & (df['time'] <= row[1]), 'year'] = row[2]
df = df[df['year'].notna()]


#df = df.set_index("time",drop=False)

In [126]:
df.head(5)

,time,tanom_cao,cao_area_weighted,t2min_weighted,year
305,1940-11-01,NaN,0.0,3.262868,1941.0
306,1940-11-02,NaN,0.0,5.909746,1941.0
307,1940-11-03,NaN,0.0,5.332471,1941.0
308,1940-11-04,NaN,0.0,4.235930,1941.0
309,1940-11-05,NaN,0.0,6.497868,1941.0


In [127]:
#CAO index
df["cao_index"] = df["tanom_cao"] * df["cao_area_weighted"]
df["cao_index"] = df["cao_index"].fillna(0)
df['cao_index'] = - df['cao_index']

In [128]:
df.head(5)

,time,tanom_cao,cao_area_weighted,t2min_weighted,year,cao_index
305,1940-11-01,NaN,0.0,3.262868,1941.0,-0.0
306,1940-11-02,NaN,0.0,5.909746,1941.0,-0.0
307,1940-11-03,NaN,0.0,5.332471,1941.0,-0.0
308,1940-11-04,NaN,0.0,4.235930,1941.0,-0.0
309,1940-11-05,NaN,0.0,6.497868,1941.0,-0.0


In [129]:
#date elements
df["month"] = df.time.dt.month
df['day'] = df.groupby('year').cumcount()
df['year_date'] = df.time.dt.strftime("%Y-01-01")
 
#df = df["month"].isin([1,2,12])
df["decade"] = 1940
decades = [i for i in range(1940, 2021,10)]
for y in decades:
    df.loc[(df.year>=y) & (df.year<y+10) ,"decade"] = y

In [132]:
df = df[df["month"].isin([1,2,12])]

In [133]:
df_yearly = df.groupby('year').agg({'time': {'min','max'},'t2min_weighted': 'mean','cao_index': 'max'})

df_yearly = df_yearly.reset_index()
df_yearly.columns = [
    col[0] if col[1] == '' else f"{col[0]}_{col[1]}" 
    for col in df_yearly.columns
]

df_yearly.head(5)

,year,time_min,time_max,t2min_weighted_mean,cao_index_max
0,1941.0,1940-12-01,1941-02-28,-7.882734,0.828291
1,1942.0,1941-12-01,1942-02-28,-9.421599,1.972937
2,1943.0,1942-12-01,1943-02-28,-8.382362,1.827657
3,1944.0,1943-12-01,1944-02-29,-7.102558,0.581237
4,1945.0,1944-12-01,1945-02-28,-8.523308,1.667457


In [135]:
df = df.round(3)

df.to_csv("data/cao_index_daily.csv",index=False,sep=";")

In [136]:
df_yearly = df_yearly.round(3)

df_yearly.to_csv("data/cao_index_yearly.csv",index=False,sep=";")